# 🧭 Project 6: The Travel Planner

**Objective:** Plan a 7-day itinerary using hierarchical planning and self-critique.

## 📖 The Planning Challenge

Complex tasks require:
- Breaking down goals into subtasks
- Balancing multiple constraints (budget, time, preferences)
- Self-correction when plans have issues

## 🎯 Architecture

```
Goal → Planner Agent → Draft Plan
                ↓
        Critic Agent → Identify Issues
                ↓
        Planner Agent → Refined Plan
                ↓
        [Repeat until acceptable]
```

In [ ]:
import os
import json
from typing import List, Dict, Any
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("✅ Setup complete!")

## Task 1: Define Planning Agent

In [ ]:
PLANNER_PROMPT = """You are an expert travel planner. Create detailed 7-day itineraries.

Consider:
- Budget constraints
- User interests
- Practical logistics (travel time, opening hours)
- Balance between activities and rest

Format your plan as JSON:
{
  "destination": "city",
  "total_budget": 0,
  "days": [
    {
      "day": 1,
      "activities": [{"time": "9:00 AM", "activity": "...", "cost": 0, "duration": "2h"}],
      "accommodation": {"name": "...", "cost": 0},
      "meals": [{"meal": "breakfast/lunch/dinner", "cost": 0}],
      "daily_cost": 0
    }
  ],
  "total_cost": 0
}
"""

def create_plan(requirements: str, feedback: str = None) -> Dict[str, Any]:
    """
    Generate travel plan based on requirements and optional feedback.
    """
    messages = [{"role": "system", "content": PLANNER_PROMPT}]
    
    if feedback:
        messages.append({
            "role": "user",
            "content": f"Previous feedback: {feedback}\n\nRevise the plan accordingly."
        })
    
    messages.append({"role": "user", "content": requirements})
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    
    try:
        plan = json.loads(response.choices[0].message.content)
        return plan
    except:
        return {"error": "Failed to parse plan"}

print("✅ Planner agent ready")

## Task 2: Define Critic Agent

In [ ]:
CRITIC_PROMPT = """You are a critical travel reviewer. Analyze travel plans for issues.

Check for:
1. Budget violations (does total_cost exceed specified budget?)
2. Unrealistic logistics (too many activities, not enough travel time)
3. Missing essentials (meals, accommodation)
4. Poor balance (too exhausting, too boring)
5. Cost accuracy (do daily costs sum correctly?)

Respond with JSON:
{
  "approved": true/false,
  "issues": ["list of problems found"],
  "suggestions": ["list of improvements"],
  "score": 0-10
}
"""

def critique_plan(plan: Dict[str, Any], requirements: str) -> Dict[str, Any]:
    """
    Critique a travel plan and provide feedback.
    """
    prompt = f"""Requirements: {requirements}

Plan to review:
{json.dumps(plan, indent=2)}

Provide detailed critique:"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": CRITIC_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3  # More deterministic for critique
    )
    
    try:
        critique = json.loads(response.choices[0].message.content)
        return critique
    except:
        return {"error": "Failed to parse critique"}

print("✅ Critic agent ready")

## Task 3: Implement Plan-Critique-Refine Loop

In [ ]:
def hierarchical_planning(requirements: str, max_iterations: int = 3) -> Dict[str, Any]:
    """
    Plan with self-correction loop.
    """
    print("="*80)
    print("🎯 Starting hierarchical planning")
    print("="*80 + "\n")
    
    plan = None
    feedback = None
    
    for iteration in range(max_iterations):
        print(f"\n🔄 Iteration {iteration + 1}/{max_iterations}")
        print("-"*80)
        
        # Generate or refine plan
        print("\n📝 Planner: Creating plan...")
        plan = create_plan(requirements, feedback)
        
        if "error" in plan:
            print(f"❌ Error: {plan['error']}")
            continue
        
        print(f"✓ Plan created for {plan.get('destination', 'unknown')}")
        print(f"  Total cost: ${plan.get('total_cost', 0)}")
        
        # Critique plan
        print("\n🔍 Critic: Reviewing plan...")
        critique = critique_plan(plan, requirements)
        
        if "error" in critique:
            print(f"❌ Error: {critique['error']}")
            break
        
        print(f"  Score: {critique.get('score', 0)}/10")
        print(f"  Approved: {critique.get('approved', False)}")
        
        if critique.get('issues'):
            print(f"  Issues found: {len(critique['issues'])}")
            for issue in critique['issues']:
                print(f"    - {issue}")
        
        # Check if approved
        if critique.get('approved', False):
            print("\n✅ Plan approved!")
            return {
                'plan': plan,
                'critique': critique,
                'iterations': iteration + 1
            }
        
        # Prepare feedback for next iteration
        feedback = f"""Issues: {', '.join(critique.get('issues', []))}
Suggestions: {', '.join(critique.get('suggestions', []))}"""
        
        print("\n🔧 Preparing for refinement...")
    
    print("\n⚠️ Maximum iterations reached")
    return {
        'plan': plan,
        'critique': critique,
        'iterations': max_iterations
    }

print("✅ Hierarchical planner ready")

## 🧪 Test the Planning System

In [ ]:
# Test scenario
requirements = """
Plan a 7-day trip to Tokyo, Japan.

Constraints:
- Budget: $2000 total (including accommodation, food, activities)
- Interests: Technology, anime culture, traditional temples, food
- Must include: Visit to TeamLab, Akihabara, at least 2 temples
- Preference: Mix of modern and traditional experiences
"""

result = hierarchical_planning(requirements, max_iterations=3)

In [ ]:
# Display final plan
print("\n" + "="*80)
print("📋 FINAL TRAVEL PLAN")
print("="*80)

plan = result['plan']
print(f"\nDestination: {plan.get('destination', 'N/A')}")
print(f"Total Budget: ${plan.get('total_budget', 0)}")
print(f"Total Cost: ${plan.get('total_cost', 0)}")
print(f"\nIterations to approval: {result['iterations']}")
print(f"Final score: {result['critique'].get('score', 0)}/10")

print("\n" + "-"*80)
print("Daily Breakdown:")
print("-"*80)

for day in plan.get('days', []):
    print(f"\nDay {day['day']}:")
    print(f"  Activities:")
    for activity in day.get('activities', []):
        print(f"    {activity.get('time', 'TBD')}: {activity.get('activity', 'N/A')} (${activity.get('cost', 0)}, {activity.get('duration', 'N/A')})")
    print(f"  Accommodation: {day.get('accommodation', {}).get('name', 'N/A')} (${day.get('accommodation', {}).get('cost', 0)})")
    print(f"  Daily Cost: ${day.get('daily_cost', 0)}")

## 🎯 Challenge: Add Execution Agent

Create an executor that simulates executing the plan and reports issues.

In [ ]:
# TODO: Implement ExecutorAgent
# - Simulates going through each day
# - Reports if activities are feasible
# - Identifies scheduling conflicts
# - Validates budget spent vs planned

# YOUR CODE HERE

## 🎓 Key Takeaways

### Hierarchical Planning:

1. **Decomposition**: Break complex goals into subtasks
2. **Critique Loop**: Self-correction improves quality
3. **Multi-criteria**: Balance competing objectives
4. **Iteration**: Refine until acceptable

### Production Extensions:

- Add real-time data (prices, availability)
- Implement Monte Carlo Tree Search for optimization
- Multiple critic personas (budget, logistics, fun)
- User feedback integration

### Next: Production!

Phase 4 covers fine-tuning, deployment, and evaluation.